[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nbakas/MachineLearning/blob/main/14-Clustering.ipynb)

# Introduction

In this notebook, we use KMeans and Hierarchical Clustering to group wines based on their chemical properties. This helps identify natural wine segments without needing quality labels. These segments can support targeted marketing by identifying wines that appeal to similar consumer preferences. They help refine pricing strategies by grouping wines of similar composition and perceived quality. In production, clustering reveals how chemical features relate to wine styles, guiding process adjustments for consistency and quality control.

PCA and t-SNE are used to reduce dimensionality and visualize the clusters in 2D. They help us better understand the structure of the data and validate the separation between wine groups by projecting high-dimensional features into a visual format. PCA highlights directions of maximum variance, while t-SNE captures non-linear relationships, making cluster patterns more visible and interpretable.

# Import the dataset

In [ ]:
!pip install ucimlrepo

In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch dataset 
# https://archive.ics.uci.edu/dataset/186/wine+quality
online_retail = fetch_ucirepo(id=186) 
  
# dataset (as pandas dataframes) 
X = online_retail.data.features 
y = online_retail.data.targets

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
import numpy as np
print(y.columns)
print(np.sort(np.unique(y.to_numpy())))

In [ ]:
import matplotlib.pyplot as plt  # Importing the matplotlib library for plotting
y_class_counts = y.value_counts()  # Counting the frequency of each class in y
plt.figure(figsize=(10, 2))  # Setting the figure size for the plot
ax = y_class_counts.plot(kind='bar')  # Creating a bar plot for class frequencies
plt.title('Frequency per y class')  # Setting the title of the plot
plt.xlabel('Class')  # Labeling the x-axis as 'Class'
plt.ylabel('Frequency')  # Labeling the y-axis as 'Frequency'
plt.xticks(rotation=0)  # Rotating x-ticks to 0 degrees for better readability
for p in ax.patches:  # Iterating over each bar in the plot
    ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),  # Annotating each bar with its height
                ha='center', va='center', xytext=(0, 5), textcoords='offset points')  # Positioning the annotation
plt.show()  # Displaying the plot

In [8]:
# We remove classes 3,9 because the number of samples is too small
# Remove classes 3 and 9
y = y.squeeze() # Convert y from DataFrame to Series
mask = ~y.isin([3, 9]) # Create a mask to keep only the rows where y is not 3 or 9

# Apply the mask using loc to keep index alignment
X = X.loc[mask] # Keep only the rows where y is not 3 or 9
y = y.loc[mask] # Keep only the rows where y is not 3 or 9

In [ ]:
print(X.shape)
print(y.shape)

In [11]:
# Dropping some columns to reduce the dimensionality of the data
X = X.drop(columns=['fixed_acidity', 'volatile_acidity', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'sulphates'])

In [ ]:
# Storing the remaining feature names
features_names = X.columns
features_names

In [ ]:
# Display the first few rows of the dataframe to understand the structure of the data
X.head()

In [15]:
# Converting the dataframe to a numpy array
X = X.to_numpy()
y = y.to_numpy()

# Scale the data

In [16]:
scaler_type = 'StandardScaler' # Select among 'StandardScaler', 'MinMaxScaler', 'MaxAbsScaler', 'NOscaling'

In [17]:
if scaler_type != 'NOscaling':
    if scaler_type == 'StandardScaler':
        from sklearn.preprocessing import StandardScaler
        # The StandardScaler will scale the data to have mean 0 and standard deviation 1
        scaler = StandardScaler()
    elif scaler_type == 'MinMaxScaler':
        from sklearn.preprocessing import MinMaxScaler
        # The MinMaxScaler will scale the data to a given range (default is 0 to 1)
        scaler = MinMaxScaler()
    elif scaler_type == 'MaxAbsScaler':
        from sklearn.preprocessing import MaxAbsScaler
        # The MaxAbsScaler will scale the data to the range [-1, 1] based on the maximum absolute value
        scaler = MaxAbsScaler()
    X = scaler.fit_transform(X)

In [26]:
def plot_features_scatter(X, y, features_names):
    # Create a grid of subplots with dimensions based on the number of features
    fig, axes = plt.subplots(len(features_names), len(features_names), figsize=(15, 15))
    # Iterate over each feature for the x-axis
    for i in range(len(features_names)):
        # Iterate over each feature for the y-axis
        for j in range(len(features_names)):
            # Check if the current subplot is not on the diagonal
            if i != j:
                # Scatter plot of the data points for the i-th and j-th features, colored by cluster labels
                scatter = axes[i, j].scatter(X[:, i], X[:, j], c=y, cmap='inferno', s=10)
                legend1 = axes[i, j].legend(*scatter.legend_elements(), title="Clusters")
                axes[i, j].add_artist(legend1)
                # Set the x-axis label to the i-th feature name
                axes[i, j].set_xlabel(features_names[i])
                # Set the y-axis label to the j-th feature name
                axes[i, j].set_ylabel(features_names[j])
                # Set the title of the subplot to show the i-th feature vs the j-th feature
                axes[i, j].set_title(f'{features_names[i]} vs {features_names[j]}')
            else:
                # Plot the histogram of the i-th feature
                axes[i, j].hist(X[:, i], bins=20)
                # Set the title of the subplot to show the i-th feature
                axes[i, j].set_title(features_names[i])
                # Set the x-axis label to the i-th feature name
                axes[i, j].set_xlabel(features_names[i])
                # Set the y-axis label to 'Frequency'
    # Adjust the layout to prevent overlap
    plt.tight_layout()
    # Display the plot
    plt.show()

In [ ]:
plot_features_scatter(X, y, features_names)

We do not observe any strong physical cluster in the pairwise plots. Hence we proceed with Clustering.

# KMeans clustering

In [ ]:
# Importing KMeans from sklearn.cluster for clustering
from sklearn.cluster import KMeans
# Import the silhouette_score function from sklearn.metrics
from sklearn.metrics import silhouette_score  

# Determine the optimal number of clusters using the silhouette score
range_n_clusters = list(range(2, 11))  # Define a range of cluster numbers to evaluate, from 2 to 10
best_n_clusters = 2  # Initialize the best number of clusters with the minimum value in the range
best_silhouette_score = -1  # Initialize the best silhouette score with a very low value

for n_clusters in range_n_clusters:  # Iterate over the range of cluster numbers
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)  # Initialize KMeans with the current number of clusters
    kmeans.fit(X)  # Fit the KMeans model to the scaled data
    cluster_labels = kmeans.labels_  # Retrieve the cluster labels assigned to each data point
    silhouette_avg = silhouette_score(X, cluster_labels)  # Calculate the average silhouette score for the current clustering
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")  # Print the silhouette score for the current number of clusters
    if silhouette_avg > best_silhouette_score:  # Check if the current silhouette score is better than the best one found so far
        best_silhouette_score = silhouette_avg  # Update the best silhouette score
        best_n_clusters = n_clusters  # Update the best number of clusters
        best_cluster_labels = kmeans.labels_  # Retrieve the cluster labels assigned to each data point
        best_cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=features_names) # This gives you a table: feature values per cluster center. 
        best_cluster_features_stds = pd.DataFrame([X[best_cluster_labels == i].std(axis=0) for i in range(best_n_clusters)], columns=features_names) # This gives you a table: std of each feature per cluster.
print(f"The optimal number of clusters is {best_n_clusters} with a silhouette score of {best_silhouette_score:.2f}")  # Print the optimal number of clusters and the corresponding silhouette score

In [ ]:
print(best_cluster_centers)

In [ ]:
print(best_cluster_features_stds)

The variables best_cluster_centers and best_cluster_features_stds are useful for understanding the characteristics of each cluster.
- best_cluster_centers: This DataFrame contains the feature values for the center of each cluster. It helps in identifying the typical or average feature values that define each cluster, providing insights into the central tendency of the data points within each cluster.
- best_cluster_features_stds: This DataFrame contains the standard deviation of each feature within each cluster. It is useful for understanding the variability or spread of the data points around the cluster centers. A lower standard deviation indicates that the data points are closely packed around the cluster center, while a higher standard deviation suggests more dispersion.


In [ ]:
plot_features_scatter(X, best_cluster_labels, features_names)

# Hierarchical clustering

In [55]:
# Import necessary functions for hierarchical clustering and silhouette score calculation
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics import silhouette_score

# Perform hierarchical clustering
# The linkage function from scipy.cluster.hierarchy is typically used for agglomerative clustering. The method='ward' specifies that the Ward's method is used to minimize the variance within each cluster.
linkage_matrix = linkage(X, method='ward')

Structure of the Linkage Matrix:

The result is a NumPy array of shape (n-1, 4), where n is the number of original observations (data points). Each row corresponds to one merge step in the clustering process. Each row has 4 columns:

| Column Index | Description                                            |
| ------------ | ------------------------------------------------------ |
| `Z[i, 0]`    | Index of the first cluster merged                      |
| `Z[i, 1]`    | Index of the second cluster merged                     |
| `Z[i, 2]`    | Distance (dissimilarity) between the merged clusters   |
| `Z[i, 3]`    | Number of original samples in the newly formed cluster |

In [ ]:
# Reorder columns: [Cluster1, Cluster2, SampleCount, Distance]
from copy import deepcopy
cols = deepcopy(linkage_matrix[:, [0, 1, 3, 2]])

# Select first and last 5 rows
subset = np.vstack([cols[:5], cols[-5:]])

# Print header
print(f"{'Cluster 1':>10} {'Cluster 2':>10} {'Count':>10} {'Distance':>10}")

# Format and print each row
for row in subset:
    c1, c2, count, dist = int(row[0]), int(row[1]), int(row[2]), float(row[3])
    print(f"{c1:10} {c2:10} {count:10} {dist:10.2f}")

In [ ]:
# Determine the optimal number of clusters using the silhouette score
range_n_clusters = list(range(2, 11))  # Define a range of cluster numbers to evaluate, from 2 to 10
best_n_clusters_hierarchical = 2  # Initialize the best number of clusters with the minimum value in the range
best_silhouette_score_hierarchical = -1  # Initialize the best silhouette score with a very low value

for n_clusters in range_n_clusters:  # Iterate over the range of cluster numbers
    cluster_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')  # Retrieve the cluster labels assigned to each data point
    silhouette_avg = silhouette_score(X, cluster_labels)  # Calculate the average silhouette score for the current clustering
    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")  # Print the silhouette score for the current number of clusters
    if silhouette_avg > best_silhouette_score_hierarchical:  # Check if the current silhouette score is better than the best one found so far
        best_cluster_labels = cluster_labels  # Update the best cluster labels
        best_silhouette_score_hierarchical = silhouette_avg  # Update the best silhouette score
        best_n_clusters_hierarchical = n_clusters  # Update the best number of clusters

print(f"The optimal number of clusters for hierarchical clustering is {best_n_clusters_hierarchical} with a silhouette score of {best_silhouette_score_hierarchical:.2f}")  

In [ ]:
plot_features_scatter(X, best_cluster_labels, features_names)

In [ ]:
# Visualize a part of the dendrogram
# Create a figure with a specific size
plt.figure(figsize=(20, 4))

# Generate a dendrogram from the linkage matrix, truncating the dendrogram to show only the last 30 merged clusters.
# p is the number of last merged clusters to display. It controls the number of displayed leaf nodes.  Each leaf may represent a single data point or a cluster of data points, depending on the total number of original data points and the value of p. p is used with the truncate_mode='lastp' parameter.
# Colors indicate different high-level clusters formed near the root of the hierarchy
nof_merged_clusters = 50    
dendrogram(linkage_matrix, truncate_mode='lastp', p=nof_merged_clusters, leaf_font_size=10, show_leaf_counts=True)

# Set the title of the dendrogram plot
plt.title('Hierarchical Clustering Dendrogram (truncated)')

# Label the x-axis as 'Sample index'
plt.xlabel(f'Clusters formed in the last {nof_merged_clusters} merges (values = number of samples in each)')

# Label the y-axis as 'Distance'
plt.ylabel('Distance') # Distance is the distance between the two clusters that are merged at each step of the hierarchical clustering.

# Display the plot
plt.show()

# Principal Component Analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA  # Import PCA for dimensionality reduction

# Fit KMeans with the optimal number of clusters
best_n_clusters = 4
kmeans = KMeans(n_clusters=best_n_clusters, random_state=42)  # Initialize KMeans with the optimal number of clusters and a random state for reproducibility
kmeans.fit(X)  # Fit KMeans to the scaled data
cluster_labels = kmeans.labels_  # Retrieve the cluster labels assigned to each data point

# Perform PCA to reduce the data to 2 dimensions for visualization
pca = PCA(n_components=2, random_state=42)  # Initialize PCA to reduce the data to 2 dimensions
X_pca = pca.fit_transform(X)  # Fit PCA to the scaled data and transform it
X_pca.shape  # Output the shape of the PCA-transformed data
# Check variance retained
variance_ratios = pca.explained_variance_ratio_
total_variance_retained = variance_ratios.sum()

print(f"Variance explained by each component: {variance_ratios}")
print(f"Total variance retained with 2 components: {total_variance_retained:.2%}")

In [ ]:
# Create a figure with a specific size for the scatter plot
plt.figure(figsize=(8, 6))

# Create a scatter plot of the PCA-reduced data
colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan'] 

# Create a scatter plot with custom colors
for cluster in range(best_n_clusters):
    # Scatter plot for each cluster
    # X_pca[cluster_labels == cluster, 0] represents the PCA-reduced data points for the current cluster on the x-axis
    # X_pca[cluster_labels == cluster, 1] represents the PCA-reduced data points for the current cluster on the y-axis
    # color=colors[cluster] assigns a unique color to each cluster
    # label=f'Cluster {cluster}' assigns a label to each cluster for the legend
    # marker='o' specifies the marker style for the scatter plot
    # s=20 sets the size of the markers in the scatter plot
    plt.scatter(X_pca[cluster_labels == cluster, 0], X_pca[cluster_labels == cluster, 1], 
                color=colors[cluster], label=f'Cluster {cluster}', marker='o', s=20)

# Set the title of the scatter plot
plt.title('KMeans Clusters Visualized using PCA')

# Label the x-axis as 'Principal Component 1'
plt.xlabel('Principal Component 1')

# Label the y-axis as 'Principal Component 2'
plt.ylabel('Principal Component 2')

# Add a legend to the plot
plt.legend()

# Display the plot
plt.show()

# t-SNE

In [ ]:
from sklearn.manifold import TSNE  # Import TSNE for t-SNE dimensionality reduction

# Perform t-SNE to reduce the data to 2 dimensions for visualization
tsne = TSNE(n_components=2, random_state=42)  # Initialize t-SNE to reduce the data to 2 dimensions
X_tsne = tsne.fit_transform(X)  # Fit t-SNE to the scaled data and transform it

# Create a figure with a specific size for the scatter plot
plt.figure(figsize=(8, 6))

# Create a scatter plot of the t-SNE-reduced data
for cluster in range(best_n_clusters):
    # Scatter plot for each cluster
    plt.scatter(X_tsne[cluster_labels == cluster, 0], X_tsne[cluster_labels == cluster, 1], 
                color=colors[cluster], label=f'Cluster {cluster}', marker='o', s=20)

# Set the title of the scatter plot
plt.title('KMeans Clusters Visualized using t-SNE')

# Label the x-axis as 't-SNE Component 1'
plt.xlabel('t-SNE Component 1')

# Label the y-axis as 't-SNE Component 2'
plt.ylabel('t-SNE Component 2')

# Add a legend to the plot
plt.legend()

# Display the plot
plt.show()